# Extract Kafka log for ratings


In [4]:
from kafka import KafkaConsumer
import re
from datetime import datetime, timedelta

TOPIC_NAME = "movielog15"
rate_pattern = re.compile(r'^.*?,\d+,GET /rate/.*?=\d+$')
end_time = datetime.now() - timedelta(days=10)
start_time = end_time - timedelta(days=1)


consumer = KafkaConsumer(
    TOPIC_NAME,
    bootstrap_servers=['localhost:9092'],
    auto_offset_reset='earliest',  # Start reading from the beginning
    enable_auto_commit=True,
    group_id='rating-group',
    value_deserializer=lambda x: x.decode('utf-8')
)
# Process and store logs
total_count = 0
with open('extracted_ratings_sample.txt', 'w') as output_file:
    for message in consumer:
        line = message.value

        # Extract the timestamp from the ConsumerRecord
        message_timestamp = message.timestamp
        message_time = datetime.fromtimestamp(message_timestamp / 1000.0)

        # Check if the message timestamp is within the desired range
        if message_time < start_time:
            print(f"Message too old.")
            continue

        if message_time > end_time:
            print(f"Message too new.")
            break

        # Write matching logs to the file
        if rate_pattern.match(line):
            total_count += 1
            output_file.write(line + '\n')
            print(f"Matched rating log: {line}")
        
        if total_count == 150000:
            break
print(f"Total logs processed: {total_count}")
print("Extracted ratings saved to 'extracted_ratings.txt'")

Matched rating log: 2024-10-14T15:20:53,69835,GET /rate/like+water+for+chocolate+1992=3
Matched rating log: 2024-10-14T15:20:53,302703,GET /rate/the+dark+knight+rises+2012=4
Matched rating log: 2024-10-14T15:20:54,212553,GET /rate/the+great+debaters+2007=5
Matched rating log: 2024-10-14T15:20:55,19606,GET /rate/the+flying+scotsman+2006=3
Matched rating log: 2024-10-14T15:20:56,470992,GET /rate/71+fragments+of+a+chronology+of+chance+1994=4
Matched rating log: 2024-10-14T15:20:57,375836,GET /rate/casino+royale+2006=4
Matched rating log: 2024-10-14T15:20:58,342438,GET /rate/forrest+gump+1994=5
Matched rating log: 2024-10-14T15:20:59,42291,GET /rate/jarhead+2005=3
Matched rating log: 2024-10-14T15:21:01,166080,GET /rate/highwaymen+2004=3
Matched rating log: 2024-10-14T15:21:01,390312,GET /rate/loggerheads+2005=4
Matched rating log: 2024-10-14T15:21:02,411376,GET /rate/the+hunger+games+mockingjay+-+part+1+2014=4
Matched rating log: 2024-10-14T15:21:03,491739,GET /rate/pusher+1996=4
Matched 

# Convert Kafka log to csv

In [10]:
import csv

with open('extracted_ratings.txt', 'r') as infile, open('extracted_ratings.csv', 'w', newline='') as outfile:
    fieldnames = ['user_time', 'user_id', 'movie_id', 'movie_title', 'year', 'rating']
    writer = csv.DictWriter(outfile, fieldnames=fieldnames)
    
    writer.writeheader()
    
    for line in infile:
        parts = line.strip().split(',')
        
        if len(parts) < 3:
            print(f"Skipping malformed line: {line}")
            continue
        
        try:
            time = parts[0]
            userid = parts[1]
            request = parts[2].strip()
            
            if not request.startswith('GET /rate/'):
                print(f"Skipping unexpected request format: {line}")
                continue
            
            movie_rating = request[len('GET /rate/'):].split('=')
            
            if len(movie_rating) < 2:
                print(f"Skipping malformed movie path: {line}")
                continue
            
            movieid = movie_rating[0]
            movie_info = movie_rating[0].split('+')
            rating = movie_rating[1]
            
            year = movie_info[-1]  
            movietitle = ' '.join(movie_info[:-1])  
            
            writer.writerow({
                'user_time': time,
                'user_id': userid,
                'movie_id': movieid,
                'movie_title': movietitle,
                'year': year,
                'rating': rating
            })
        
        except Exception as e:
            print(f"Error processing line: {line}, Error: {str(e)}")
            continue


# Extract movie details from API

In [ ]:
import requests
import pandas as pd

def get_movie_details(movie_id):
    response = requests.get(f"http://128.2.204.215:8080/movie/{movie_id}")
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data for movie_id: {movie_id}")
        return None

def process_movie_details(movie_info):
    genres = ', '.join([genre['name'] for genre in movie_info.get('genres', [])])
    production_companies = ', '.join([company['name'] for company in movie_info.get('production_companies', [])])
    production_countries = ', '.join([country['name'] for country in movie_info.get('production_countries', [])])
    spoken_languages = ', '.join([lang['name'] for lang in movie_info.get('spoken_languages', [])])

    return {
        'movie_id': movie_info['id'],
        'tmdb_id': movie_info.get('tmdb_id', 'N/A'),
        'imdb_id': movie_info.get('imdb_id', 'N/A'),
        'title': movie_info.get('title', 'N/A'),
        'original_title': movie_info.get('original_title', 'N/A'),
        'adult': movie_info.get('adult', 'N/A'),
        'budget': movie_info.get('budget', 'N/A'),
        'genres': genres,
        'homepage': movie_info.get('homepage', 'N/A'),
        'original_language': movie_info.get('original_language', 'N/A'),
        'overview': movie_info.get('overview', 'N/A'),
        'popularity': movie_info.get('popularity', 'N/A'),
        'poster_path': movie_info.get('poster_path', 'N/A'),
        'production_companies': production_companies,
        'production_countries': production_countries,
        'release_date': movie_info.get('release_date', 'N/A'),
        'revenue': movie_info.get('revenue', 'N/A'),
        'runtime': movie_info.get('runtime', 'N/A'),
        'spoken_languages': spoken_languages,
        'status': movie_info.get('status', 'N/A'),
        'vote_average': movie_info.get('vote_average', 'N/A'),
        'vote_count': movie_info.get('vote_count', 'N/A')
    }

movie_ids = pd.read_csv('movie_ids.csv')['movie_id'].tolist()

movie_data = []

for movie_id in movie_ids:
    movie_info = get_movie_details(movie_id)
    if movie_info:
        processed_data = process_movie_details(movie_info)
        movie_data.append(processed_data)

movie_df = pd.DataFrame(movie_data)
movie_df.to_csv('movie_details.csv', index=False)

print("Movie details saved to 'movie_details.csv'")


# Extract User detail from API

In [ ]:
import requests
import pandas as pd

def get_user_details(user_id):
    response = requests.get(f"http://128.2.204.215:8080/user/{user_id}")
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to retrieve data for user_id: {user_id}")
        return None

def process_user_details(user_info):
    return {
        'user_id': user_info['user_id'],
        'age': user_info.get('age', 'N/A'),
        'occupation': user_info.get('occupation', 'N/A'),
        'gender': user_info.get('gender', 'N/A')
    }

user_ids = pd.read_csv('user_ids.csv')['user_id'].tolist()

user_data = []

for user_id in user_ids:
    user_info = get_user_details(user_id)
    if user_info:
        processed_data = process_user_details(user_info)
        user_data.append(processed_data)

user_df = pd.DataFrame(user_data)
user_df.to_csv('user_details.csv', index=False)

print("User details saved to 'user_details.csv'")
